# A surpreendente instrução `match`

## Terminologia

A instrução `match` recebe uma expressão, chamada **sujeito**.

Cada cláusula `case` define um **padrão** e um bloco de código.

In [1]:
sujeito = 4j

def analisar(sujeito):
    
    match sujeito:
    
        case 42:  # padrão: constante
            print('caso 0: 42\n\tsujeito == 42')
    
        case ['OK', x]:  # padrão: sequência de 'OK' mais um item qualquer
            
            print(f'caso 1: {sujeito}\n\t{x=}')
            
        case [int(a), int(b)]: # padrão: sequência formada dois inteiros
            
            print(f'caso 2: {sujeito}\n\t{a=} {b=}')
            
        case {'peso': float(p)}:  # padrão: qualquer mapeamento com uma chave `'peso'` associada a um float 
            
            print(f'caso 3: {sujeito}\n\t{p=}')
            
        case complex(real=0, imag=y):  # padrão: número imaginário (real=0)
            
            print(f'caso 4: {sujeito}\n\t{y=}')
            
        case _:
            
            print(f'caso coringa: {sujeito}')


for sujeito in [
    42,
    ('OK', 200),
    [60, 45],
    {'peso': 70.},
    3.3j,
    {'peso': 3},
]:
    analisar(sujeito)

caso 0: 42
	sujeito == 42
caso 1: ('OK', 200)
	x=200
caso 2: [60, 45]
	a=60 b=45
caso 3: {'peso': 70.0}
	p=70.0
caso 4: 3.3j
	y=3.3
caso coringa: {'peso': 3}


## Funcionamento

Python compara o **sujeito** ao **padrão** em cada `case`.

Assim que encontra um padrão correspondente, Python executa o bloco daquele `case`.

Se o padrão tem variáveis, elas são atribuidas como variáveis locais no bloco.

Quando um padrão corresponde, só aquele `case` é executado. Os seguintes são ignorados.

Se nenhum padrão corresponde, nada acontece (como um `while False:`).

## Estrutura de um padrão

Um padrão pode ser formado por:

* sequências;
* mapeamentos;
* instâncias;
* alternativas separadas por `|`;
* variáveis;
* coringa `_`.

## Comparação com `switch`

A instrução `match` pode ser usada para comparar o **sujeito** a valores literais simples, como um `switch` em outras linguagens. 

Porém, cada **padrão** pode representar uma estrutura complexa,
contendo variáveis no lugar de algumas partes.
Através dessas variáveis podemos obter partes do **sujeito** declarativamente,
sem escrever código para navegar pela estrutura proc

Chama-se **desestruturar** o ato de obter as partes da estrutura do **sujeito**.


## Padrões sequência

### Exemplo: registros de cidades

Esse exemplo mostra a desestruturação de sequências aninhadas (sequências dentro de sequências).

In [2]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
for record in metro_areas:
    match record:  # (1)
        case [name, _, _, (lat, lon)] if lon <= 0:  # (2)
            print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')

                |  latitude | longitude
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
São Paulo       |  -23.5478 |  -46.6358


### Exemplo: interpretador lis.py

[código-fonte](https://github.com/lispylab/pybr24/blob/52d15b411ae212ea2afe2c00eac77f30b2928965/lis.py#L151)

## Surpresa 0: Delimitadores [] e () são equivalentes

Os padrões `[a, (b, c)]` e `(a, [b, c])` são equivalentes.

Na sintaxe de padrões sequências podem ser expressas com `[]` ou `()`. Tanto faz.

## Surpresa 1: Tratamento diferente para certas sequências

O `case` não desestrutura sequências `str`, `bytes` e `bytearray`.
Concretamente: não é possível obter um caractere específico de uma `str` em um `case`.

A regra é: podem ser desestruturadas instâncias de sub-classes de `collections.Sequence`, exceto `str`, `bytes`, e `bytearray`.

Tipos sequência da biblioteca padrão que pode ser desestruturadas com `match/case`:

```
list     memoryview    array.array
tuple    range         collections.deque
```


## Surpresa 2: Uso de variáveis

Variáveis como `x` em `int(x)` podem ser criadas e atribuídas em cada `case`, assim como `for i in range(9):` cria e atribui valor a `i`.

Se você precisa que parte de um padrão seja um valor esperado porém variável, é obrigatório usar um identificador com ponto (*dotted name*), como `Cores.VERMELHO`. Se usar apenas `VERMELHO`, o identificador será considerado uma variável a ser atribuída se o padrão casar.

In [3]:
from enum import Enum

VERMELHO = '#F00'

class Cores(Enum):
    VERMELHO = 1
    VERDE = 2
    AZUL = 3

sujeito = [Cores.VERDE]

match sujeito:
    case [VERMELHO]:
        print(f'caso 1: {sujeito}')
    case [Cores.VERDE]:
        print(f'caso 2: {sujeito}')

caso 1: [<Cores.VERDE: 2>]


## Surpresa 3: casamento parcial em sequências, mapeamentos e instâncias

Sequências sempre são comparadas por inteiro. Para casar com itens sobrando, use `*resto` ou `*_`.

Mapeamentos são comparados somente pelas chaves informadas.
Para corresponder, o sujeito deve casar todas as chaves que aparecem no padrão.
Chaves sobrando no sujeito são irrelevantes.

Para instâncias com atributos posicionais ou nomeados, vale a mesma ideia dos mapeamentos:
Para corresponder, o sujeito deve casar todos os atributos que aparecem no padrão.
Atributos sobrando no sujeito são irrelevantes.


## Surpresa 4: `int(x)` não é o que parece

`int(x)` não significa converter `x` em `int`, mas sim verificar que `x` é uma instância de `int`.

Na sintaxe de um padrão, `f(x)` não significa `f.__call__(x)`.

Além disso, a atribuição a `x` segue três regras diferentes!

### Regra 1: instâncias capturadas como um todo

Para esses tipos **abençoados**, a variável engloba o objeto inteiro, e não apenas um atributo:

```
bool   bytearray   bytes   dict   float   frozenset   int   list   set   str   tuple
```

### Regra 2: instâncias desestruturadas por variáveis posicionais
Se a classe contém um atributo `__match_args__`, ela suporta padrões com argumentos posicionais.

O valor de `__match_args__` define quais atributos são casados por posição, e em qual ordem.

As fábricas de classes `collections.namedtuple`, `typing.NamedTuple`, e `@dataclasses.dataclass`
criam o `__match_args__` automaticamente:


In [4]:
from typing import NamedTuple

class Coordenada(NamedTuple):
    lat: float
    lon: float

Coordenada.__match_args__ 

('lat', 'lon')

In [5]:
sp = Coordenada(-23.6, -46.6)

match sp:
    case Coordenada(lat=y, lon=x):
        print(x, y)

-46.6 -23.6


Em classes feitas na unha, você cria o `__match_args__` se quiser a conveniência de escrever
padrões com argumentos posicionais:

In [6]:
from typing import NamedTuple

class Pet:
    nome: str
    peso: float
    __match_args__ = 'nome', 'peso'

    def __init__(self, nome, peso):
        self.nome = nome
        self.peso = peso

fido = Pet('Fido', 7.5)

match fido:
    case Pet(x, y):
        print(x, y)

Fido 7.5


### Regra #3: instâncias desestruturadas por variáveis nomeadas

Quando não é **abençoada** (Regra #1), nem tem `__match_args__` (Regra #2),
o tipo pode ser desestruturado usando a sintaxe de argumentos nomeados
para acessar os atributos do tipo.


In [7]:
match fido:
    case Pet(nome=n, peso=p):
        print(n, p)

Fido 7.5


### Resumo: os 3 sentidos de `c(x)`

In [8]:
def analisar(sujeito):
    match sujeito:
        case str(x): # qualquer string, x captura a string toda
            print(f'Regra #1: {sujeito}\n\t{x=}')
        case Coordenada(x):  # qualquer Coordenada, x captura o atributo `lat` por posição
            print(f'Regra #2: {sujeito}\n\t{x=}')
        case complex(real=x):  # qualquer complexo, x captura o atributo `real` por nome
            print(f'Regra #3: {sujeito}\n\t{x=}')

for sujeito in [
    'oxente',
    Coordenada(23, 46),
    3.3+4.4j
]:
    analisar(sujeito)



Regra #1: oxente
	x='oxente'
Regra #2: Coordenada(lat=23, lon=46)
	x=23
Regra #3: (3.3+4.4j)
	x=3.3


## Surpresa 5: operador `|` não invoca `__or__`

Quando usado em `case`, o operador `|` indica um padrão alternativo.

Fora desse contexto, a expressão `a | b` sempre invoca `a.__or__(b)`, exceto dentro de um `case`.